# Map Wrapper Development for Scipy Differential Equation

Proof of concept notebook for first Scipy, since it only requires map not the o

Error_function is a pass function.
Our "map" takes care of initiating the whole neuron group, calculating the errors and returing the value.

In [1]:
import numpy as np
import multiprocessing as mp

from scipy.optimize import differential_evolution, rosen
from scipy.optimize._differentialevolution import DifferentialEvolutionSolver


In [2]:
from brian2 import *
from brian2.equations.equations import (DIFFERENTIAL_EQUATION, Equations,
                                        SingleEquation, PARAMETER)
from brian2.input import TimedArray
from brian2 import NeuronGroup, StateMonitor, store, restore, run, defaultclock, second, Quantity
from brian2.stateupdaters.base import StateUpdateMethod



### Input / Output

In [3]:
input_traces = zeros((10,5))*volt
for i in range(5):
    input_traces[5:,i]=i*10*mV

In [4]:
# Create target current traces
output_traces = 10*nS*input_traces

In [5]:
input = input_traces
output = output_traces

### Parameters 

In [6]:
input_var = 'v'
output_var = 'I'

parameter_names = {'g', 'E'}
method = ('linear', 'exponential_euler', 'euler')
t_start = 0*second
popsize = 1
dt = 0.1 *ms
defaultclock.dt = dt

### Calculation for inner parameters

In [7]:
Nsteps, Ntraces = input_traces.shape
N = popsize * len(parameter_names)
duration = Nsteps*dt

In [8]:
N

2

### Setup The Model for Optimization
```
params, fits, error = fit_traces(model = model, input_var = 'v', output_var = 'I',\
                                 input = input_traces, output = output_traces,
                                 dt = 0.1*ms, g = [1*nS, 30*nS], E = [-20*mV,100*mV],
                                 tol = 1e-6)
```

In [9]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [10]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

INFO       No numerical integration method specified, using method 'linear' (took 0.01s). [brian2.stateupdaters.base.method_choice]


### Required Model Operations

In [11]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
# input_traces = TimedArray(input, dt = dt)

In [12]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

In [13]:
neurons = NeuronGroup(Ntraces*N, model, method = method)
neurons.namespace['input_var'] = input_traces
neurons.namespace['output_var'] = output_traces
neurons.namespace['t_start'] = t_start
neurons.namespace['Ntraces'] = Ntraces

# Record error
neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                      when='end')

# Add the code doing the numerical integration
neurons.run_regularly(state_update_code, when='groups')

store()


In [14]:
def error_function(params):
        # Set parameter values, duplicated over traces
        # params is a list of vectors (vector = value for population)
        d = dict()
        for name, value in zip(parameter_names, params.T):
            d[name] = (value * ones((Ntraces,N))).T.flatten()

        # Run the model
        restore()
        neurons.set_states(d, units = False)
        run(duration, namespace = {})

        e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
        e = mean(e.reshape((N,Ntraces)),axis=1)
        return array(e)

### Dev Map Optimization 

#### Artificial Data

In [15]:
params = np.array([
 [ 1.80869973e-08,  2.50218013e-02],
 [ 1.88373085e-08,  9.89559934e-02]])

params.T

array([[1.80869973e-08, 1.88373085e-08],
       [2.50218013e-02, 9.89559934e-02]])

In [16]:
parameter_names

{'E', 'g'}

In [17]:
# def error_map_function(func, iterable, *args):
def error_map_function(params):
    # Set parameter values, duplicated over traces
    # params is a list of vectors (vector = value for population)
    d = dict()
    for name, value in zip(parameter_names, params.T):
        d[name] = (value * ones((Ntraces,N))).T.flatten()
            
    restore()
    
    #### NEURONS SET_STATES HAS TO BE FIXED TO SET IT FOR ALL 
    
    neurons.set_states(d, units = False)
    run(duration, namespace = {})

    e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
    e = mean(e.reshape((N,Ntraces)),axis=1)
    return array(e)
    
#     return d
#     pass


In [18]:
error_map_function(params)

BrianObjectException: Original error and traceback:
Traceback (most recent call last):
  File "/home/alteska/Desktop/brian/brian2/build/lib.linux-x86_64-3.6/brian2/core/network.py", line 864, in before_run
    obj.before_run(run_namespace)
  File "/home/alteska/Desktop/brian/brian2/build/lib.linux-x86_64-3.6/brian2/groups/neurongroup.py", line 880, in before_run
    self.equations.check_units(self, run_namespace=run_namespace)
  File "/home/alteska/Desktop/brian/brian2/build/lib.linux-x86_64-3.6/brian2/equations/equations.py", line 945, in check_units
    user_identifiers=external)  # all variables are user defined
  File "/home/alteska/Desktop/brian/brian2/build/lib.linux-x86_64-3.6/brian2/groups/group.py", line 739, in resolve_all
    run_namespace=run_namespace)
  File "/home/alteska/Desktop/brian/brian2/build/lib.linux-x86_64-3.6/brian2/groups/group.py", line 696, in _resolve
    return self._resolve_external(identifier, run_namespace=run_namespace)
  File "/home/alteska/Desktop/br

### Optimization Init

In [ ]:
bounds = [(1, 30), (-20, 100)]

In [ ]:
differential_evolution(test, bounds, updating='deferred', workers=mp.Pool(2).map, maxiter=10)